In [2]:
import pandas as pd


In [3]:
data = pd.read_excel(r"C:\Users\hwade\OneDrive\Desktop\github\kv-transformer\Aufgabe_Abrechnung.xlsx", sheet_name = "Input",header=1)
data.iloc[4:,0] = pd.to_datetime(data.iloc[4:,0])
data.set_index(data.iloc[0:,0], inplace=True)
data.iloc[1,5] = pd.to_datetime('2019-12-13 00:00:00')


In [4]:
def RLM_Rechnung(preis, nf, penalty=False):
    bill = (nf*preis).sum()
    if penalty:
        total_hours = len(preis)
        pro_Jahr = total_hours / 365
        if pro_Jahr >= 7000 and (preis.stack().nlargest(1000) / preis.sum()).max() > 0.12:
            bill *= 1.2 
    return bill


In [82]:
def SLP_Rechnung(preis, nf):
    return (nf*preis).sum()



[1, 3, 4, 7, 8]

In [87]:

for idx, column in enumerate(data.columns[2:]):  
    typ = data[column].iloc[0]
    lieferbeginn = pd.to_datetime(data[column].iloc[1])
    lieferende = pd.to_datetime(data[column].iloc[2])
    idx_lb = data.index.get_loc(lieferbeginn)
    idx_le = data.index.get_loc(lieferende)
    nf = data[column].iloc[idx_lb:idx_le+1]
    preis = data.iloc[idx_lb:idx_le+1,1]
    zeitraum = pd.date_range(start = lieferbeginn, end = lieferende, freq = 'h')
    zeit_data = pd.DataFrame(range(len(zeitraum)), index = zeitraum )
    we_stunden = [ zeit_data.index.dayofweek >= 0] #& (zeit_data.index.hour >= 20)] #and (zeit_data.index.hour < 8)) 
    

[ True  True  True ...  True  True  True]
[ True  True  True ...  True  True  True]
[ True  True  True ...  True  True  True]
[ True  True  True ...  True  True  True]
[ True  True  True ...  True False False]
[ True  True  True ...  True  True  True]
[ True  True  True ... False False False]
[False False False ...  True  True  True]
[ True  True  True ... False False False]
[False False False ... False False False]


In [68]:
def SLPHTNT_Rechnung(preis, nf):
    zeitraum = pd.date_range(start = lieferbeginn, end = lieferende, freq = 'h')
    zeit_data = pd.DataFrame(range(len(zeitraum)), index = zeitraum )
    we_stunden = (zeit_data.index.dayofweek >= 4) & ((zeit_data.index.hour >= 20) & (zeit_data.index.hour < 8))
    we_bill = ((nf*preis)[we_stunden]).sum() * (0.9)
    return we_bill


In [69]:
for idx, column in enumerate(data.columns[2:]):  
    typ = data[column].iloc[0]
    lieferbeginn = pd.to_datetime(data[column].iloc[1])
    lieferende = pd.to_datetime(data[column].iloc[2])
    idx_lb = data.index.get_loc(lieferbeginn)
    idx_le = data.index.get_loc(lieferende)
    nf = data[column].iloc[idx_lb:idx_le+1]
    preis = data.iloc[idx_lb:idx_le+1,1]
    if typ == 'RLM':
        bill = RLM_Rechnung(preis, nf, penalty=True)
    elif typ == 'SLP':
        bill = SLP_Rechnung(preis, nf)
    elif typ == 'SLP HT' or typ == 'SLP NT':
        if idx == 7:  
            bill = SLPHTNT_Rechnung_sub(preis, nf) 
        else:
            bill = SLPHTNT_Rechnung(preis, nf)
    print(f"Rechnung für {column}: {bill/1000}")

Rechnung für K1: 19606.890386338673
Rechnung für K2: 0.0
Rechnung für K3: 30358.540875013452
Rechnung für K4: 5760.358197935899
Rechnung für K5: 40485.5278754199
Rechnung für K6: 18935.65319591628
Rechnung für K7: 28591.533392225432
Rechnung für K8: 7787.408165881716
Rechnung für K9: 20960.281077370208
Rechnung für K10: 39117.910119866836


In [46]:

def SLPHTNT_Rechnung_sub(preis, nf):
    zeitraum = pd.date_range(start = lieferbeginn, end = lieferende+pd.Timedelta(hours=1), freq = 'h')
    zeit_data = pd.DataFrame(range(len(zeitraum)), index = zeitraum )
    we_stunden = (zeit_data.index.dayofweek >= 4) & ((zeit_data.index.hour >= 20) & (zeit_data.index.hour < 8))
    we_bill = (nf*preis[we_stunden]).sum() * (0.9)
    return (nf*preis).sum() +we_bill


In [50]:
for idx, column in enumerate(data.columns[2:]):  
    typ = data[column].iloc[0]
    lieferbeginn = pd.to_datetime(data[column].iloc[1])
    lieferende = pd.to_datetime(data[column].iloc[2])
    idx_lb = data.index.get_loc(lieferbeginn)
    idx_le = data.index.get_loc(lieferende)
    nf = data[column].iloc[idx_lb:idx_le+1]
    preis = data.iloc[idx_lb:idx_le+1,1]
    if typ == 'RLM':
        bill = RLM_Rechnung(preis, nf, penalty=True)
    elif typ == 'SLP':
        bill = SLP_Rechnung(preis, nf)
    elif typ == 'SLP HT' or typ == 'SLP NT':
        if idx == 7:  
            bill = SLPHTNT_Rechnung_sub(preis, nf) 
        else:
            bill = SLPHTNT_Rechnung(preis, nf)
    print(f"Rechnung für {column}: {bill/1000}")


Rechnung für K1: 19606.890386338673
Rechnung für K2: 18575.6846825873
Rechnung für K3: 30358.540875013452
Rechnung für K4: 5760.358197935899
Rechnung für K5: 40485.5278754199
Rechnung für K6: 18935.65319591628
Rechnung für K7: 28591.533392225432
Rechnung für K8: 7787.408165881716
Rechnung für K9: 20960.281077370208
Rechnung für K10: 39117.910119866836


In [75]:
def SLPHTNT_Rechnung_sub(preis, nf):
    zeitraum = pd.date_range(start = lieferbeginn, end = lieferende+pd.Timedelta(hours=1), freq = 'h')
    zeit_data = pd.DataFrame(range(len(zeitraum)), index = zeitraum )
    we_stunden = (zeit_data.index.dayofweek >= 4) & ((zeit_data.index.hour >= 20) & (zeit_data.index.hour < 8))
    we_bill = (nf*preis[we_stunden]).sum() * (0.9)
    return zeit_data.index.dayofweek
for idx, column in enumerate(data.columns[2:]):  
    typ = data[column].iloc[0]
    lieferbeginn = pd.to_datetime(data[column].iloc[1])
    lieferende = pd.to_datetime(data[column].iloc[2])
    idx_lb = data.index.get_loc(lieferbeginn)
    idx_le = data.index.get_loc(lieferende)
    nf = data[column].iloc[idx_lb:idx_le+1]
    preis = data.iloc[idx_lb:idx_le+1,1]
    if typ == 'SLP HT' or typ == 'SLP NT':
        if idx == 7:  
            bill = SLPHTNT_Rechnung_sub(preis, nf) 
        else:
            pass
            #bill = SLPHTNT_Rechnung(preis, nf)
    print(bill)

1
1
1
1
1
1
1
Index([2, 2, 2, 2, 3, 3, 3, 3, 3, 3,
       ...
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
      dtype='int32', length=3602)
Index([2, 2, 2, 2, 3, 3, 3, 3, 3, 3,
       ...
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
      dtype='int32', length=3602)
Index([2, 2, 2, 2, 3, 3, 3, 3, 3, 3,
       ...
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
      dtype='int32', length=3602)
